# CS 584 :: Data Mining :: George Mason University :: Spring 2024


# Homework 3: Clustering

- **100 points [8% of your final grade]**
- **Due Sunday, March 31 by 11:59pm**

- *Goals of this homework:* (1) implement your K-means model; (2) implement K-means++; (3) Apply PCA to K-Means.

- *Submission instructions:* for this homework, you only need to submit to Blackboard. Please name your submission **FirstName_Lastname_hw3.ipynb**, so for example, my submission would be something like **Ziwei_Zhu_hw3.ipynb**. Your notebook should be fully executed so that we can see all outputs.

## Part 1: K-Means (60 points)

In this part, you will implement your own K-means algorithm to conduct clustering on handwritten digit images. In this homework, we will still use the handwritten digit image dataset we have already used in previous homework. However, since clustering is unsupervised learning, which means we do not use the label information anymore. So, here, we will only use the testing data stored in the "test.txt" file.

First, let's load the data by excuting the following code:

In [17]:
import numpy as np

test = np.loadtxt("test.txt", delimiter=',')
test_features = test[:, 1:]
test_labels = test[:, 0]
print('array of testing feature matrix: shape ' + str(np.shape(test_features)))
print('array of testing label matrix: shape ' + str(np.shape(test_labels)))

array of testing feature matrix: shape (10000, 784)
array of testing label matrix: shape (10000,)


Now, it's time for you to implement your own K-means algorithm. First, please write your code to build your K-means model using the image data with **K = 10**, and **Euclidean distance**.

**Note: You should implement the algorithm by yourself. You are NOT allowed to use Machine Learning libraries like Sklearn**

**Note: you need to decide when to stop the iteration.**

In [18]:
# Write your code here

import random

random_id_val = np.random.choice(len(test_features),10,replace=False)
centroids = [ test_features[i] for i in random_id_val ]
itr = 0
old_centroids = None

while np.not_equal(centroids,old_centroids).any():
    clusters = [[] for i in range(10)]
    for test_sample in test_features:
        distance = np.sqrt(np.sum((test_sample - centroids)**2, axis=1))
        clusters[np.argmin(distance)].append(test_sample)
    old_centroids = centroids
    centroids = [np.mean(cluster, axis=0) for cluster in clusters]
    for i, centroid in enumerate (centroids):
        if np.isnan(centroid).any():
            centroids[i] = old_centroids[i]

Next, you need to calculate the square root of Sum of Squared Error (SSE) of each cluster generated by your K-means algorithm. Then, print out the averaged SSE of your algorithm.

In [19]:
# Write your code here
clust_sse = []

for i in range(10):
    sse =0
    for sample in clusters[i]:
        sse += np.sum((sample - centroids[i])**2)
    clust_sse.append(np.sqrt(sse))

avg_sse = np.mean(clust_sse)
print("Square root of SSE for each cluster:")
for i, sse in enumerate(clust_sse):
  print (f"Cluster {i:1}: {sse}")

print(f"\nAverage SSE: {avg_sse}")

Square root of SSE for each cluster:
Cluster 0: 37960.512352599784
Cluster 1: 55895.28703055209
Cluster 2: 50984.530095863716
Cluster 3: 59854.798019528665
Cluster 4: 49752.04764722419
Cluster 5: 51780.90429753062
Cluster 6: 50890.42663484296
Cluster 7: 54384.91897866432
Cluster 8: 40558.438083125315
Cluster 9: 48342.94296953176

Average SSE: 50040.48061094634


Then, please have a look on https://scikit-learn.org/stable/modules/generated/sklearn.metrics.homogeneity_completeness_v_measure.html#sklearn.metrics.homogeneity_completeness_v_measure, and use this function to print out the homogeneity, completeness, and v-measure of your K-means model.

In [20]:
# Write your code here

from sklearn.metrics import homogeneity_completeness_v_measure

# Assigning labels to clusters (labels_pred)
labels_pred = np.zeros(len(test_labels))
for i, cluster in enumerate(clusters):
    for sample in cluster:
        index = np.where((test_features == sample).all(axis=1))[0][0]
        labels_pred[index] = i

homogeneity, completeness, v_measure = homogeneity_completeness_v_measure(test_labels, labels_pred)

print(f'Homogeneity: {homogeneity}')
print(f'Completeness: {completeness}')
print(f'V-measure: {v_measure}')


Homogeneity: 0.5044867648538972
Completeness: 0.516158347413129
V-measure: 0.510255820968763


## Part 2: K-Means++ (30 points)

Ok, now you already have a good model. But can you further improve it? In the next cell, please implement the K-means++ model introduced in the lecture.

**Note: Everything else is the same as Part1, i.e., K=10, and you need to use Euclidean distance.**

In [21]:
import numpy as np
import random

def kmeans_plus_plus_centroids(data, K=10):
    initial_centroid_idx = np.random.choice(len(data))
    centroids = [data[initial_centroid_idx]]

    for _ in range(1, K):
        distances = np.array([min([np.sum((x - centroid)**2) for centroid in centroids]) for x in data])
        probabilities = distances / distances.sum()
        cumulative_probabilities = probabilities.cumsum()
        r = np.random.rand()

        for i, p in enumerate(cumulative_probabilities):
            if r < p:
                centroids.append(data[i])
                break

    return centroids

K = 10
centroids = kmeans_plus_plus_centroids(test_features, K)

itr = 0
old_centroids = None

while np.not_equal(centroids,old_centroids).any():
    clusters = [[] for _ in range(K)]
    for test_sample in test_features:
        distance = np.sqrt(np.sum((test_sample - centroids)**2, axis=1))
        clusters[np.argmin(distance)].append(test_sample)
    old_centroids = centroids
    centroids = [np.mean(cluster, axis=0) for cluster in clusters if len(cluster) > 0]
    for i, centroid in enumerate(centroids):
        if np.isnan(centroid).any():
            centroids[i] = old_centroids[i]


In the next cell, use sklearn.metrics.homogeneity_completeness_v_measure() to print out the homogeneity, completeness, and v-measure of your K-means++ model.

In [22]:
from sklearn.metrics import homogeneity_completeness_v_measure

labels_pred = np.zeros(len(test_features))  # Initialize array to hold predicted labels
for cluster_id, cluster in enumerate(clusters):
    for sample in cluster:
        # Find the index of this sample in the original dataset
        index = np.where((test_features == sample).all(axis=1))[0][0]
        labels_pred[index] = cluster_id  # Assign cluster ID as the predicted label

homogeneity, completeness, v_measure = homogeneity_completeness_v_measure(test_labels, labels_pred)

print(f'Homogeneity: {homogeneity}')
print(f'Completeness: {completeness}')
print(f'V-measure: {v_measure}')

Homogeneity: 0.47769987588798835
Completeness: 0.48050235939018887
V-measure: 0.47909701938430954


## Part 3: Dimension Reduction by PCA (10 points)

Last, in this part, please resue your code of PCA from HW1 to reduce the feature dimension here. And then, apply your K-Means code to the reduced data and report homogeneity, completeness, and v-measure.

**Note: You need to consider the reduced dimension m of PCA as a hyper-parameter to tune. That is, you need to try different m and measure the corresponding clustering performance. At the end, you need to report the best m and clustering performance.**

**Note: Everything else is the same as Part1, i.e., K=10, and you need to use Euclidean distance.**

In [23]:
# Write your code here

from sklearn.metrics import homogeneity_completeness_v_measure
import numpy as np

class PCA:
    def __init__(self, n_components):
        self.n_components = n_components
        self.components = None
        self.mean = None

    def fit(self, X):
        self.mean = np.mean(X, axis=0)
        X_centered = X - self.mean

        _, _, v = np.linalg.svd(X_centered, full_matrices=False)

        self.components = v[:self.n_components]

    def transform(self, X):
        X_centered = X - self.mean
        return np.dot(X_centered, self.components.T)

# Instantiate PCA and fit to training data
pca = PCA(100)
pca.fit(test_features)

# Transform the training data
test_features_pca = pca.transform(test_features)
print('Shape of transformed training data:', test_features_pca.shape)

# K-Means algorithm code
random_id_val = np.random.choice(len(test_features_pca), 10, replace=False)
centroids = [test_features_pca[i] for i in random_id_val]
itr = 0
old_centroids = None

while np.not_equal(centroids, old_centroids).any():
    clusters = [[] for i in range(10)]
    for test_sample in test_features_pca:
        distance = np.sqrt(np.sum((test_sample - centroids)**2, axis=1))
        clusters[np.argmin(distance)].append(test_sample)
    old_centroids = centroids
    centroids = [np.mean(cluster, axis=0) for cluster in clusters]
    for i, centroid in enumerate(centroids):
        if np.isnan(centroid).any():
            centroids[i] = old_centroids[i]

# Code for SSE and Average SSE
clust_sse = []

for i in range(10):
    sse = 0
    for sample in clusters[i]:
        sse += np.sum((sample - centroids[i])**2)
    clust_sse.append(np.sqrt(sse))

avg_sse = np.mean(clust_sse)
print("Square root of SSE for each cluster:")
for i, sse in enumerate(clust_sse):
    print(f"Cluster {i}: {sse}")

print(f"\nAverage SSE: {avg_sse}")

# Assigning labels to clusters (labels_pred)
labels_pred = np.zeros(len(test_labels))
for i, cluster in enumerate(clusters):
    for sample in cluster:
        index = np.where((test_features_pca == sample).all(axis=1))[0][0]
        labels_pred[index] = i

homogeneity, completeness, v_measure = homogeneity_completeness_v_measure(test_labels, labels_pred)

print(f'Homogeneity: {homogeneity}')
print(f'Completeness: {completeness}')
print(f'V-measure: {v_measure}')


Shape of transformed training data: (10000, 100)
Square root of SSE for each cluster:
Cluster 0: 46961.36884687147
Cluster 1: 50277.58179942565
Cluster 2: 49332.93775344256
Cluster 3: 54834.137235201386
Cluster 4: 35080.218930415154
Cluster 5: 57992.121049033456
Cluster 6: 32352.686638317973
Cluster 7: 46586.92224534605
Cluster 8: 47361.20172114954
Cluster 9: 47834.895917437825

Average SSE: 46861.40721366411
Homogeneity: 0.5037010963293469
Completeness: 0.5069607455373373
V-measure: 0.5053256643220212


### Question: What is the best reduced dimension m from your experiment? What is the correpsonding clustering performance (homogeneity, completeness, and v-measure)?


Test 1 : m=50, Homogeneity:0.51, Completeness:0.52, V-measure:0.51

Test 2 : m=10, Homogeneity:0.47, Completeness:0.48, V-measure:0.48

Test 3 : m=20, Homogeneity:0.51, Completeness:0.52, V-measure:0.52

Test 4 : m=30, Homogeneity:0.49, Completeness:0.50, V-measure:0.49

Test 5 : m=40, Homogeneity:0.50, Completeness:0.51, V-measure:0.50

Test 6 : m=100, Homogeneity:0.51, Completeness:0.53, V-measure:0.52

Test 7 : m=150, Homogeneity:0.51, Completeness:0.51, V-measure:0.51

Best reduced dimension is m=100, clustering performance is:

Homogeneity: 0.5146

Completeness: 0.5377

V-measure: 0.5262

